# SCRAPPED

In [1]:
###
### PYTHON PACKAGES
###
import pandas as pd
import re
import emoji
from ftfy import fix_text 
import unidecode
from langdetect import detect, detect_langs
import spacy
from spacy import displacy

###
### USER-DEVELOPED IMPORTS
###
# from project import *
from capture_group import *
from output_format import *

In [26]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']
win_data = pd.read_csv('wins.csv')['text']
spacy_data = pd.read_csv('spacy_info.csv')

In [45]:
from itertools import islice

account_rts = {}

for text in cleaned_data:
    tweet = text.split(' ')
    if tweet[0] == 'RT':
        if tweet[1] in account_rts:
            account_rts[tweet[1]] += 1
        else:
            account_rts.update({tweet[1]:1})

account_dicts = dict(sorted(account_rts.items(), key=lambda item: item[1], reverse=True))

top_accounts = list(islice(account_dicts, 20))
top_accounts

print(top_accounts)

['@goldenglobes:', '@eonline:', '@PerezHilton:', '@TheEllenShow:', '@EmWatson:', '@nbcsnl:', '@CNNshowbiz:', '@VanityFair:', '@EW:', '@BuzzFeed:', '@vulture:', '@nbc:', '@piersmorgan:', '@MARLONLWAYANS:', '@HuffingtonPost:', '@buckhollywood:', '@MarilynMonroeES:', '@DavidSpade:', '@THR:', '@MTVNews:']


In [46]:
text = "RT @goldenglobes: Best Actor in a Motion Picture - Comedy or Musical - Hugh Jackman (@RealHughJackman) - Les Miserables - #GoldenGlobes"

In [47]:
retweet_pattern = r'RT\s' + '|'.join(top_accounts)
reputable_df = cleaned_data.apply(lambda text: text if re.search(retweet_pattern, text, re.IGNORECASE) else None)

In [48]:
reputable_df.dropna(inplace=True)

In [49]:
keys = r"(\bwin\b|\bwins\b|\bwon\b|\bawarded\b|\baward\b|\bprize\b|\bhonor\b|\bmedal\b|\btrophy\b|\bbest)"
reputable_df = reputable_df.apply(lambda x: x if re.search(keys, x.lower()) != None else None)
reputable_df.dropna(inplace=True)

In [50]:
reputable_df.to_csv("reputable_sources_only.csv")

In [51]:
text = 'RT @goldenglobes: Best Actor in a Motion Picture - Comedy or Musical - Hugh Jackman (@RealHughJackman) - Les Miserables - #GoldenGlobes'

In [52]:
if re.search(keys, text.lower()) != None:
    print(text)
else:
    print("fail")

RT @goldenglobes: Best Actor in a Motion Picture - Comedy or Musical - Hugh Jackman (@RealHughJackman) - Les Miserables - #GoldenGlobes


In [53]:
results = reputable_df.apply(find_award_winner)

In [54]:
results.dropna(inplace=True)

In [55]:
results.apply(extract_answers)

12350     None
12370     None
12373     None
12459     None
12610     None
          ... 
152151    None
152600    None
152741    None
152749    None
152850    None
Name: text, Length: 1090, dtype: object

In [57]:
top_awards = dict(sorted(awards_list.items(), key=lambda item: item[1].votes, reverse=True))
for i in top_awards:
    print(top_awards[i].output())

{'best actor in a motion picture': {'nominees': [], 'presenters': [], 'winner': [('hugh jackman', 128), ('daniel day lewis', 18)], 'votes': 146}}
{'best song at the': {'nominees': [], 'presenters': [], 'winner': [('adele', 144)], 'votes': 144}}
{'best supporting actress in a motion picture is': {'nominees': [], 'presenters': [], 'winner': [('anne hathaway', 114), ('sudah ku', 2), ('yayyyy', 2)], 'votes': 118}}
{'best director motion picture': {'nominees': [], 'presenters': [], 'winner': [('goldenglobe', 102)], 'votes': 102}}
{'best director': {'nominees': [], 'presenters': [], 'winner': [('affleck', 2), ('ben affleck', 74)], 'votes': 76}}
{'best motion picture': {'nominees': [], 'presenters': [], 'winner': [('les miserables', 39), ('lesmis', 4), ('goldenglobes', 10), ('argo', 12), ('the film', 4)], 'votes': 69}}
{'best actor': {'nominees': [], 'presenters': [], 'winner': [('damien lewis', 42), ('hugh jackman', 2), ('daniel day lewis', 22), ('film wajib tonton ini', 1)], 'votes': 67}}
{

In [73]:
award_names = top_awards.keys()
award_names

dict_keys(['best actor in a motion picture', 'best song at the', 'best supporting actress in a motion picture is', 'best director motion picture', 'best director', 'best motion picture', 'best actor', 'best director category', 'best animated feature film is', 'best actress in tv drama', 'best original score', 'best picture', 'best supporting actress', 'best comedy series', 'best motion picture drama', 'best drama at the', 'best actress in a drama at the', 'best actor in a drama for lincoln at the', 'best supporting actor at', 'best actor at', 'best actress in a motion picture', 'best actress at the', 'best supporting actress in a motion picture win for les miserables', 'best supporting actor in', 'if i ever won an', 'best original song for skyfall', 'best tv series', 'best original song in a motion picture', 'best original song at the', 'best actor in a tv series', 'best actor in a musical', 'best screenplay for', 'best director for', 'best supporting actor globe for', 'best original s

### HANDLING TWEET TIMING

In [367]:
tweets_df = pd.read_csv("text_cleaned.csv")


In [368]:
def create_award_pattern(award_name):
    # Convert to lowercase
    award_name = award_name.lower()
    # Replace punctuation with spaces
    award_name = re.sub(r'[^\w\s]', ' ', award_name)
    # Split into words
    words = award_name.split()
    # Remove common stopwords
    stopwords = {'in', 'a', 'the', 'of', '–', '-', 'or'}
    words = [word for word in words if word not in stopwords]
    # Create regex pattern with optional spaces and words
    pattern = r'\b' + r'\s*'.join(words) + r'\b'
    # Replace spaces with \s* to allow for any whitespace
    pattern = pattern.replace(' ', r'\s*')
    return pattern

In [369]:
award_patterns = {award: create_award_pattern(award) for award in list(award_names)}

In [370]:
# Compile regex patterns
compiled_award_patterns = {award: re.compile(pattern) for award, pattern in award_patterns.items()}

# Initialize a dictionary to hold mentions
award_mentions = {award: [] for award in award_names}

# Iterate over tweets
for index, row in tweets_df.iterrows():
    tweet_text = row['text']
    position = index
    # Check each award
    for award, pattern in compiled_award_patterns.items():
        if pattern.search(tweet_text):
            award_mentions[award].append(position)

In [371]:
import numpy as np

award_stats = {}

for award, positions in award_mentions.items():
    if positions:
        mean_pos = np.mean(positions)
        std_pos = np.std(positions)
        award_stats[award] = {
            'mean': mean_pos,
            'std': std_pos,
            'positions': positions
        }
    else:
        award_stats[award] = {
            'mean': None,
            'std': None,
            'positions': []
        }

In [390]:
for award, stats in award_stats.items():
    if stats['positions']:
        mean = stats['mean']
        std = stats['std']
        lower_bound = mean - 50
        upper_bound = mean + 50
        within_range = [
            pos for pos in stats['positions'] if lower_bound <= pos <= upper_bound
        ]
        stats['range'] = (lower_bound, upper_bound)
        stats['positions_within_range'] = within_range
    else:
        stats['range'] = (None, None)
        stats['positions_within_range'] = []


In [391]:
for award, stats in award_stats.items():
    print(f"Award: {award}")
    if stats['positions']:
        print(f"Mean Position: {stats['mean']}")
        print(f"Standard Deviation: {stats['std']}")
        print(f"Range within 1 std: {stats['range']}")
    else:
        print("No mentions found.")
    print("-" * 40)


Award: best actor in a motion picture
No mentions found.
----------------------------------------
Award: best song at the
Mean Position: 49730.0
Standard Deviation: 12150.149217190708
Range within 1 std: (49680.0, 49780.0)
----------------------------------------
Award: best supporting actress in a motion picture is
No mentions found.
----------------------------------------
Award: best director motion picture
No mentions found.
----------------------------------------
Award: best director
Mean Position: 120405.20673076923
Standard Deviation: 13827.895084635458
Range within 1 std: (120355.20673076923, 120455.20673076923)
----------------------------------------
Award: best motion picture
Mean Position: 126809.02150537634
Standard Deviation: 26769.476385148468
Range within 1 std: (126759.02150537634, 126859.02150537634)
----------------------------------------
Award: best actor
Mean Position: 115148.79093198992
Standard Deviation: 36200.58761403089
Range within 1 std: (115098.7909319899

In [392]:
nlp = spacy.load('en_core_web_sm')

In [393]:
def extract_entities(text, award_labels=['PERSON', 'WORK_OF_ART', 'ORG', 'GPE']):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ in award_labels:
            entities.append(ent.text)
    
    if entities == []:
        return None
    return entities

In [394]:
import wikipediaapi
wiki = wikipediaapi.Wikipedia('student_homework')

def lookup_wikipedia(name, role):
   page = wiki.page(name)

   if page.exists():
      return re.search(role, page.summary, re.IGNORECASE) != None 
   else:
      return False

In [ ]:
candidates = {}
for award, stats in award_stats.items():
    candidates.update({award:{}})
    tweets_within_range = stats['range']
    if tweets_within_range != (None, None):
        temp_tweets = tweets_df[(tweets_df.index > int(tweets_within_range[0])) & (tweets_df.index < int(tweets_within_range[1]))]['text']
        
        actor = "actor" in award
        actress = "actress" in award
        director = "director" in award
        
        ignore = set()
        
        if actor or actress or director:
            all_ppl = temp_tweets.apply(lambda x: extract_entities(x, ['PERSON'])).dropna()
            for i in all_ppl:
                curr = i[0]
                if curr in candidates[award]:
                    candidates[award][curr] += 1
                elif "#" in curr or "@" in curr:
                    ignore.add(curr)
                elif curr not in ignore:
                    if actor and lookup_wikipedia(curr, "Actor"):
                        candidates[award].update({curr:1})
                    elif actress and lookup_wikipedia(curr, "Actress"):
                        candidates[award].update({curr:1})
                    elif director and lookup_wikipedia(curr, "Director"):
                        candidates[award].update({curr:1})
                    else:
                        ignore.add(curr)

In [399]:
candidates

{'best actor in a motion picture': {},
 'best song at the': {},
 'best supporting actress in a motion picture is': {},
 'best director motion picture': {},
 'best director': {'Ben Affleck': 3,
  'Jodi Foster': 2,
  'Chad Lowe': 1,
  'Jodie Foster': 6,
  'Lena Dunham': 2},
 'best motion picture': {},
 'best actor': {'Ben Affleck': 19,
  'Josh Brolin': 4,
  'Moonrise Kingdom': 4,
  'Bill Murray': 2,
  'Ben Afleck': 1,
  'Tina Fey': 2,
  'P.T. Anderson': 1,
  'Tommy Lee Jones': 1},
 'best director category': {'Jodie Foster': 5,
  'Ben Affleck': 2,
  'Tarantino': 1,
  'Chad Lowe': 1,
  'Lena Dunham': 1},
 'best animated feature film is': {},
 'best actress in tv drama': {},
 'best original score': {},
 'best picture': {},
 'best supporting actress': {'Kristen Wiig': 2,
  'Anne Hathaway': 10,
  'Vanessa Hudgens': 1,
  'Amanda Seyfried': 2,
  'Jennifer Lawrence': 1},
 'best comedy series': {},
 'best motion picture drama': {},
 'best drama at the': {},
 'best actress in a drama at the': {},
